In [29]:
import wikipedia as wiki
from datetime import datetime
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer

In [42]:
base_url = 'https://www.nyasatimes.com'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

response = requests.get('https://www.nyasatimes.com/rights-activists-press-on-chakwera-to-resign-over-chizuma-saga/', headers=headers)

soup = BeautifulSoup(response.content, 'lxml')

In [45]:
mydivs = soup.find_all("div", {"class": "nyasa-content"})

In [47]:
len(mydivs)

1

In [46]:
texts = [

[<div class="nyasa-content">
 <div dir="auto">Some rights activists are putting pressure on President Dr. Lazarus McCarthy Chakwera to resign over the handling of the Anti-Corruption Bureau (ACB) director Martha Chizuma’s interdiction saga.</div>
 <div dir="auto"></div>
 <div dir="auto">Government interdicted Chizuma this week over a leaked audio clip and this has gone down well both locally and internationally with the US threatening to cut aid as this has been interpreted as the government is shielding corrupt people.<a href="https://www.nyasatimes.com/wp-content/uploads/FB_IMG_1675942210684.jpg"><img alt="" class="wp-image-209485 aligncenter lazyload" data-lazy-="" data-sizes="(max-width: 700px) 100vw, 700px" data-src="https://www.nyasatimes.com/wp-content/plugins/lazy-load/images/1x1.trans.gif" data-srcset="https://www.nyasatimes.com/wp-content/uploads/FB_IMG_1675942210684-600x450.jpg 600w, https://www.nyasatimes.com/wp-content/uploads/FB_IMG_1675942210684-300x225.jpg 300w, https:/

In [33]:
for link in soup.find_all('a', href=True):
    url = link['href']
    title = link.text
    title_len = len(title)
    if title_len > 20:
        try:
            response = requests.get(url, headers=headers)
            this_soup = BeautifulSoup(response.content, 'html.parser')
            

#content Skip to content
http://www.nyasatimes.com/ Home
https://www.nyasatimes.com/category/national/ National
https://www.nyasatimes.com/category/politics/ Politics
https://www.nyasatimes.com/category/sports/ Sports
https://www.nyasatimes.com/category/business/ Business
https://www.nyasatimes.com/category/columns/ Columns
https://www.nyasatimes.com/category/education/ Education
https://www.nyasatimes.com/category/entertainment/ Entertainment
https://www.nyasatimes.com/category/health/ Health
https://www.nyasatimes.com/category/news/ News
https://www.nyasatimes.com/category/religious-news/ Religious News
https://www.nyasatimes.com/contact-us/ Contact us
https://twitter.com/NyasaTimess 
https://www.facebook.com/nyasatimesNewspaper/ 
#  

http://www.nyasatimes.com/ Home
https://www.nyasatimes.com/category/national/ National
https://www.nyasatimes.com/category/politics/ Politics
https://www.nyasatimes.com/category/sports/ Sports
https://www.nyasatimes.com/category/business/ Business
http

In [2]:
output_dir = Path("../DATA/wiki-downloads/")
metadata_csv = output_dir.joinpath("metadata.csv")
country_list =  Path("../DATA/raw/ISO-country-list.csv")

In [3]:

cols = ['title', 'original_title', 'url', 'num_categories',
        'date_accessed', 'search_term', 'filename']

In [4]:
def download_wiki_pages(search_terms_list, 
                        max_files_per_co=100, results_per_search=10000):
    data = []
    
    for co_name in search_terms_list:
        try:
            files_saved = 0
            search_results = wiki.search(co_name, results=results_per_search)
            for result in search_results:
                try:
                    pg = wiki.page(result)
                    title = pg.title
                    date = datetime.now().date()
                    num_categories = len(pg.categories)

                    # Write page to file
                    fname = title.replace(" ", "_")
                    outf = output_dir.joinpath("{}.txt".format(fname))
                    pg_content = pg.content
                    with open(outf, "w") as text_file:
                        text_file.write(pg_content)
                    item = {'title': title, 'original_title': pg.original_title, 
                            'num_categories': num_categories, 'date_accessed': date,
                            'search_term': search_term, 'filename': outf}
                    data.append(item)
                    files_saved += 1
                    print(files_saved, co_name)
                    if files_saved > max_files_per_co:
                        break
                except:
                    pass
            if files_saved > max_files_per_co:
                break
        except:
            pass
    return data

In [5]:
def get_africa_country_list(ISO_file):
    
    df_co = pd.read_csv(country_list)
    africa_co = list(df_co.query('Continent == "Africa"')['Country'].values)
    
    return africa_co

In [6]:
df_co = pd.read_csv(country_list)
co_list = get_africa_country_list(ISO_file=country_list)

In [7]:
results = download_wiki_pages(search_terms_list=co_list[5:], 
                              max_files_per_co=10, 
                              results_per_search=50)
df = pd.DataFrame(results)
df.to_csv(metadata_csv, index=False)

/Users/dmatekenya/opt/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/dmatekenya/opt/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
